In [ ]:
%pip install mediapipe

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic  # This is bringing in the holistic model (used to make detections)
mp_drawing = mp.solutions.drawing_utils # This is bringing in the drawing utilities (used for drawing the detection)

In [ ]:
DATA_PATH = os.path.join('D:\SignLanguageData', 'MP_Data2')
VIDEO_PATH = os.path.join(DATA_PATH, 'Videos')
action = 'hello'

In [ ]:
# Define the video codec and output file name
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_file = os.path.join(VIDEO_PATH, action ,'output.mp4')

# Get the webcam input
avi_cap = cv2.VideoCapture(0)

# Get the video properties from the webcam input
frame_width = int(avi_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(avi_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(avi_cap.get(cv2.CAP_PROP_FPS))

# Calculate the number of frames to record for 1 minute
duration = 60  # 1 minute
num_frames = duration * fps

# Create a VideoWriter object to save the video
avi_out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Start recording
start_time = time.time()
frame_count = 0

while frame_count < num_frames:
    # Read a frame from the webcam
    ret, frame = avi_cap.read()

    # Break the loop if the video capture is unsuccessful
    if not ret:
        break

    # Write the frame to the output file
    avi_out.write(frame)

    # Display the frame
    cv2.imshow('Webcam Recording', frame)

    # Increment the frame count
    frame_count += 1

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Break the loop if the duration has exceeded 1 minute
    if time.time() - start_time > duration:
        break

# Release the resources
avi_cap.release()
avi_out.release()
cv2.destroyAllWindows()


In [ ]:
avi_cap.release()

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    image.flags.writeable = False       # image writable status is false 
    results = model.process(image)      # image is the frame from OpenCV, make predictions
    image.flags.writeable = True        # image writeable status is true
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion back to BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # draw pose landmarks 
    
    # draw left hand landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), # color landmarks
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) # color dots 
    
    # draw right hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), # color landmarks
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) # color dots 

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ lh, rh])

In [ ]:
# Get the list of mp4 files in the VIDEO_PATH directory

path = os.path.join(VIDEO_PATH, action)
# os.remove(path)
mp4_files = [file for file in os.listdir(path) if file.endswith('.mp4')]
counter = 0
# Play each mp4 file
for mp4_file in mp4_files:
    # Create the full file path
    file_path = os.path.join(path, mp4_file)
    
    # Open the video file
    video_cap = cv2.VideoCapture(file_path)
    
    # Play the video
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        os.makedirs(os.path.join(DATA_PATH, action))
        while True:
            ret, frame = video_cap.read()
            if not ret:
                break

            
            
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(frame, results)
            cv2.imshow('Video Playback', frame)
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action , str(counter))
            np.save(npy_path, keypoints)
            counter+=1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    # Release the video capture
    video_cap.release()
    cv2.destroyAllWindows()


In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    counter=0
    # NEW LOOP
    # Loop through actions
    while True:
        # Loop through sequences aka videos
        # os.mkdir(f'C://Users//Yuvraj Sahni//Documents//Test//MP_Data//{action}//{counter}')
        os.makedirs(os.path.join(DATA_PATH, action, str(counter)))
            # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

                # Read feed
            ret, frame = cap.read()

                # Make detections
            image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
            draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, counter), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, counter), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action , str(counter), str(frame_num))
            np.save(npy_path, keypoints)
            

            #     # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        counter+=1
        # if cv2.waitKey(10) & 0xFF == ord('s'):
        #     break
    cap.release()
    cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()